In [6]:
import time
import cv2
import numpy as np

def show_instructions(frame, text):
    """Overlay text instructions on the frame"""
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, text, (30, 50), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "Calibration will capture in:", (30, 90), font, 0.7, (0, 255, 255), 1, cv2.LINE_AA)

def get_hand_distance_with_timer(cap0, cap1, model, P0, P1, prompt, countdown=10):
    start_time = time.time()
    captured_distance = None
    while True:
        ret0, frame0 = cap0.read()
        ret1, frame1 = cap1.read()
        if not ret0 or not ret1:
            print("ERORR")
            continue

        # Optionally flip frames horizontally if your cameras are mirrored
        frame0 = cv2.flip(frame0, 1)
        frame1 = cv2.flip(frame1, 1)

        # Calculate remaining countdown time
        elapsed = time.time() - start_time
        time_left = int(countdown - elapsed)

        results0 = model(frame0)
        results1 = model(frame1)

        combined = np.hstack((results0[0].plot(), results1[0].plot()))
        show_instructions(combined, prompt)

        # Show countdown timer on the frame
        if time_left > 0:
            cv2.putText(combined, f"{time_left} seconds", (30, 130), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        else:
            # Capture once timer finishes
            kpts0 = results0[0].keypoints.xy[0].cpu().numpy() if len(results0[0].keypoints) > 0 else None
            kpts1 = results1[0].keypoints.xy[0].cpu().numpy() if len(results1[0].keypoints) > 0 else None

            if kpts0 is not None and kpts1 is not None and len(kpts0) >= 11 and len(kpts1) >= 11:
                lh0 = kpts0[9]
                lh1 = kpts1[9]
                if not np.allclose(lh0, [0, 0]) and not np.allclose(lh1, [0, 0]):
                    pts0 = np.array([[lh0[0]], [lh0[1]]], dtype=np.float64)
                    pts1 = np.array([[lh1[0]], [lh1[1]]], dtype=np.float64)
                    point_4d = cv2.triangulatePoints(P0, P1, pts0, pts1)
                    point_3d = (point_4d[:3] / point_4d[3]).flatten()
                    captured_distance = np.linalg.norm(point_3d)
                    cv2.putText(combined, f"Captured distance: {captured_distance:.2f}", (30, 180),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                    cv2.imshow("Calibration", combined)
                    cv2.waitKey(2000)  # Show for 2 seconds
                    break
                else:
                    cv2.putText(combined, "Hands not detected properly", (30, 180),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            else:
                cv2.putText(combined, "Hands not detected properly", (30, 180),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        cv2.imshow("Calibration", combined)
        if cv2.waitKey(1) & 0xFF == 27:  # ESC to abort calibration
            print("Calibration aborted.")
            captured_distance = None
            break

    return captured_distance

def calibrate_depth_range_with_timer(cap0, cap1, model, P0, P1):
    near = get_hand_distance_with_timer(cap0, cap1, model, P0, P1, "Step 1: Put your hands close to the cameras", countdown=15)
    if near is None:
        return None, None
    far = get_hand_distance_with_timer(cap0, cap1, model, P0, P1, "Step 2: Put your hands far away from the cameras", countdown=5)
    if far is None:
        return None, None

    cv2.destroyWindow("Calibration")
    print(f"Calibration completed. z_min={near:.2f}, z_max={far:.2f}")
    return min(near, far), max(near, far)


import cv2
import numpy as np
from ultralytics import YOLO
import socket
import json
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# ---- STEP 1: Estimate Extrinsics (R, t) once ----
def estimate_camera_pose(cap0, cap1, K):
    ret0, img0 = cap0.read()
    ret1, img1 = cap1.read()
    if not ret0 or not ret1:
        raise RuntimeError("Failed to capture calibration images")

    # Flip the images horizontally to correct mirroring
    img0 = cv2.flip(img0, 1)
    img1 = cv2.flip(img1, 1)

    gray0 = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)


    orb = cv2.ORB_create(1000)
    kp0, des0 = orb.detectAndCompute(gray0, None)
    kp1, des1 = orb.detectAndCompute(gray1, None)

    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(des0, des1)
    matches = sorted(matches, key=lambda x: x.distance)
    matches = matches[:100]

    pts0 = np.float32([kp0[m.queryIdx].pt for m in matches])
    pts1 = np.float32([kp1[m.trainIdx].pt for m in matches])

    E, _ = cv2.findEssentialMat(pts0, pts1, K, method=cv2.RANSAC, threshold=1.0)
    _, R, t, _ = cv2.recoverPose(E, pts0, pts1, K)
    return R, t

def normalize_point(x, y, width, height):
    norm_x = (x / width) * 2 - 1
    norm_y = (y / height) * 2 - 1
    return [norm_x, -norm_y]



def normalize_depth(distance, z_min=2.0, z_max=3.0):
    # Linear map: z_min -> 0, z_max -> 1, allow outside range
    return (distance - z_min) / (z_max - z_min)

# Camera intrinsics
K0 = np.array([[395.16709009, 0, 320],
               [0, 395.16709009, 240],
               [0, 0, 1]])
K1 = np.array([[393.05858722, 0, 320],
               [0, 393.05858722, 240],
               [0, 0, 1]])

P0 = K0 @ np.hstack((np.eye(3), np.zeros((3, 1))))  # Camera 0 projection matrix [I|0]

model = YOLO("yolo11n-pose.pt")
cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)
if not cap0.isOpened() or not cap1.isOpened():
    raise RuntimeError("Cameras could not be opened.")

print("Estimating extrinsics...")
R, t = estimate_camera_pose(cap0, cap1, K0)
P1 = K1 @ np.hstack((R, t))  # Camera 1 projection matrix [R|t]
print("Rotation:\n", R)
print("Translation:\n", t.T)


z_min, z_max = calibrate_depth_range_with_timer(cap0, cap1, model, P0, P1)
if z_min is None or z_max is None:
    raise RuntimeError("Depth calibration failed or aborted.")

def normalize_depth(distance):
    return (distance - z_min) / (z_max - z_min)


def normalize_point(x, y, width, height):
    norm_x = (x / width) * 2 - 1
    norm_y = (y / height) * 2 - 1
    return [norm_x, -norm_y]

# Setup socket for Godot connection
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(('localhost', 5050))
sock.listen(1)
print("Waiting for Godot to connect...")
conn, addr = sock.accept()
print(f"Connected by {addr}")



while True:
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    if not ret0 or not ret1:
        print("Failed to grab frames")
        break

    # Flip frames to correct mirroring
    frame0 = cv2.flip(frame0, 1)
    frame1 = cv2.flip(frame1, 1)

    height0, width0 = frame0.shape[:2]
    height1, width1 = frame1.shape[:2]

    results0 = model(frame0)
    results1 = model(frame1)

    # Extract keypoints: (N, 2) numpy arrays
    kpts0 = results0[0].keypoints.xy[0].cpu().numpy() if len(results0[0].keypoints) > 0 else None
    kpts1 = results1[0].keypoints.xy[0].cpu().numpy() if len(results1[0].keypoints) > 0 else None

    # Default output for hands if missing
    left_hand_out = [0, 0, 0]
    right_hand_out = [0, 0, 0]

    if kpts0 is not None and kpts1 is not None and len(kpts0) >= 11 and len(kpts1) >= 11:
        # Get left and right hand keypoints
        lh0 = kpts0[9]
        rh0 = kpts0[10]
        lh1 = kpts1[9]
        rh1 = kpts1[10]

        # Check for valid detections (non-zero)
        def valid_kp(kp): return not np.allclose(kp, [0,0])

        # Triangulate left hand if valid in both cameras
        if valid_kp(lh0) and valid_kp(lh1):
            pts0 = np.array([[lh0[0]], [lh0[1]]], dtype=np.float64)
            pts1 = np.array([[lh1[0]], [lh1[1]]], dtype=np.float64)
            point_4d = cv2.triangulatePoints(P0, P1, pts0, pts1)
            point_3d = (point_4d[:3] / point_4d[3]).flatten()
            dist = np.linalg.norm(point_3d)
            z_norm = normalize_depth(dist)
            x_norm, y_norm = normalize_point(lh0[0], lh0[1], width0, height0)
            left_hand_out = [x_norm, y_norm, z_norm]

        # Triangulate right hand if valid in both cameras
        if valid_kp(rh0) and valid_kp(rh1):
            pts0 = np.array([[rh0[0]], [rh0[1]]], dtype=np.float64)
            pts1 = np.array([[rh1[0]], [rh1[1]]], dtype=np.float64)
            point_4d = cv2.triangulatePoints(P0, P1, pts0, pts1)
            point_3d = (point_4d[:3] / point_4d[3]).flatten()
            dist = np.linalg.norm(point_3d)
            z_norm = normalize_depth(dist)
            x_norm, y_norm = normalize_point(rh0[0], rh0[1], width0, height0)
            right_hand_out = [x_norm, y_norm, z_norm]

    data = json.dumps({'left': left_hand_out, 'right': right_hand_out})
    try:
        conn.sendall(data.encode('utf-8') + b'\n')
    except BrokenPipeError:
        print("Godot disconnected.")
        break

    # Optional: Show combined annotated frames side by side with lines for hands
    annotated0 = results0[0].plot()
    annotated1 = results1[0].plot()
    combined = np.hstack((annotated0, annotated1))

    # Draw lines between corresponding hands (optional visualization)
    for idx, (pt0, pt1) in [(9, (lh0, lh1)), (10, (rh0, rh1))]:
        if valid_kp(pt0) and valid_kp(pt1):
            pt0_int = tuple(map(int, pt0))
            pt1_int = (annotated0.shape[1] + int(pt1[0]), int(pt1[1]))
            cv2.line(combined, pt0_int, pt1_int, (0, 255, 0), 2)
            # Draw circles with radius based on distance if you want

    cv2.imshow("Triangulated Pose with Distance", combined)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

conn.close()
cap0.release()
cap1.release()
cv2.destroyAllWindows()


Estimating extrinsics...
Rotation:
 [[    0.99824  -0.0028832   -0.059198]
 [   0.018505     0.96404      0.2651]
 [   0.056305    -0.26572      0.9624]]
Translation:
 [[   -0.99238   -0.097948   -0.074716]]

0: 480x640 1 person, 17.4ms
Speed: 1.8ms preprocess, 17.4ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.3ms
Speed: 1.1ms preprocess, 11.3ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 20.5ms
Speed: 2.1ms preprocess, 20.5ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 14.8ms
Speed: 2.0ms preprocess, 14.8ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.9ms
Speed: 2.0ms preprocess, 9.9ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.6ms
Speed: 2.1ms preprocess, 9.6ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 10.2ms
Speed: 2.6m